<a href="https://colab.research.google.com/github/DanielCorreiaReis/tech-challenge-fase3/blob/main/tech_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-usyfhin9/unsloth_e5264a52a65a47e79e2b6cd9ea15e0f8
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-usyfhin9/unsloth_e5264a52a65a47e79e2b6cd9ea15e0f8
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 11.3 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [3]:
def process_dataset(item):
  return {
      "input": f"PLEASE PROVIDE ME A DESCRIPTION FOR THIS PRODUCT TITLE.\n[|Title|] {item['title']} [|eTitle|]\n\n\[|Description|] {item['content']} [|eDescription|]"
  }

In [5]:
import json
processed_data = []

with open(r'/content/drive/MyDrive/FIAP/trn.json', 'r', encoding='utf-8') as file:
  for line in file:
    item = json.loads(line)
    # Verifica se os campos title e content existem e não estão vazios
    if item.get('title') and item.get('content'):
          processed_data.append(process_dataset(item))

output_file = r'/content/drive/MyDrive/FIAP/amazon_titles_search.json'
with open(output_file, 'w', encoding='utf-8') as file:
  json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Todos os dados reformatados foram salvos em '{output_file}'.")

Todos os dados reformatados foram salvos em '/content/drive/MyDrive/FIAP/amazon_titles_search.json'.


In [6]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
model_name = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=4096,  # Defina conforme sua necessidade
    dtype=None,  # Deixa o UnsLoth escolher o melhor tipo
    load_in_4bit=True  # Carrega o modelo em 4 bits para otimização
)

# Configurar token especial para evitar erros
tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [8]:
# this_titles = "[Girls Ballet Tutu Neon Pink, Mog's Kittens, Girls Ballet Tutu Neon Blue, The Prophet, Rightly Dividing the Word]"
this_titles = "Girls Ballet Tutu Neon Pink"
prompt = (
    "You are an expert product description writer. "
    "Write a detailed and engaging product description for the following product title:\n"
    f"Title: {this_titles}\n"
    "Description: "
)

In [9]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [10]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_length=150,  # Aumenta o tamanho máximo da resposta
        do_sample=True,  # Permite variação nas respostas
        temperature=0.7,  # Controla a aleatoriedade (0.7 é um valor equilibrado)
        top_p=0.9,  # Aplica amostragem de núcleo
        repetition_penalty=1.5,  # Penaliza repetições excessivas
        eos_token_id=tokenizer.eos_token_id  # Evita respostas infinitas
    )


response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Response before training:", response)

Response before training: You are an expert product description writer. Write a detailed and engaging product description for the following product title:
Title: Girls Ballet Tutu Neon Pink
Description: 100% polyester, machine washable; no ironing needed.
This tutus is made from soft stretchy material that will keep you comfortable while dancing! It has elastic at waistband to fit most dancers comfortably without slipping off!
The bright colors make it easy on stage or in recital when all eyes need attention where they should be focused -on YOU during performance time!!!
We believe our neon pink color choice helps stand out among other performers too so everyone can see who's performing best today!! And finally but not least comes into play here....the price point we've set ourselves


In [11]:
with open("/content/drive/MyDrive/FIAP/response_before_training.txt", "w") as f:
    f.write(response)

In [13]:
# Importando dependencias para preparar modelo pra teste
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/FIAP/amazon_titles_search.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/FIAP/formatted_amazon_titles_search_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


In [14]:
# Treinamento do modelo
def format_dataset_into_model_input(data_path):
  # Load the data from the JSON file
  with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

  def separete_text(full_text):
    title_start = full_text.find("[|Title|]") + len("[|Title|]")
    title_end = full_text.find("[|eTitle|]")

    content_start = full_text.find("[|Description|]") + len("[|Description|]")
    content_end = full_text.find("[|eDescription|]")

    instruction = full_text.split('\n')[0]
    title = full_text[title_start:title_end].strip()
    response = full_text[content_start:content_end].strip()  # Fixed: Using content_start and content_end here

    return instruction, title, response

  instructions = []
  inputs = []
  outputs = []

  # Assuming your data structure has 'input' at the top level
  for prompt in data:  # Accessing the list of prompts
    if "input" in prompt:  # Checking if 'input' key exists
        instruction, title, content = separete_text(prompt["input"])
        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)
    else:
        print("Warning: 'input' key not found in item:", prompt)

  formatted_data = {
      "instruction": instructions,
      "input": inputs,
      "output": outputs
  }

  with open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:
    json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

  print(f"Todos os dados foram salvos em '{OUTPUT_PATH_DATASET}'.")

In [15]:
def format_dataset_into_model_input(data_path):
  # Load the data from the JSON file
  with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

  def separete_text(full_text):
    title_start = full_text.find("[|Title|]") + len("[|Title|]")
    title_end = full_text.find("[|eTitle|]")

    content_start = full_text.find("[|Description|]") + len("[|Description|]")
    content_end = full_text.find("[|eDescription|]")

    instruction = full_text.split('\n')[0]
    title = full_text[title_start:title_end].strip()
    response = full_text[content_start:content_end].strip()  # Fixed: Using content_start and content_end here

    return instruction, title, response

  instructions = []
  inputs = []
  outputs = []

  # Assuming your data structure has 'input' at the top level
  for prompt in data:  # Accessing the list of prompts
    if "input" in prompt:  # Checking if 'input' key exists
        instruction, title, content = separete_text(prompt["input"])
        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)
    else:
        print("Warning: 'input' key not found in item:", prompt)

  formatted_data = {
      "instruction": instructions,
      "input": inputs,
      "output": outputs
  }

  with open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:
    json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

  print(f"Todos os dados foram salvos em '{OUTPUT_PATH_DATASET}'.")

In [16]:
format_dataset_into_model_input(DATA_PATH)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Todos os dados foram salvos em '/content/drive/MyDrive/FIAP/formatted_amazon_titles_search_data.json'.
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, Features, Value

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/FIAP/formatted_amazon_titles_search_data.json"

# Define the features of your dataset explicitly
features = Features({
    'instruction': Value(dtype='string'),
    'input': Value(dtype='string'),
    'output': Value(dtype='string'),
})

dataset = load_dataset('json', data_files=OUTPUT_PATH_DATASET, split='train', features=features)
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

In [20]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 80,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="none"  # Essa linha desativa W&B
    ),
)

Map (num_proc=2):   0%|          | 0/1390403 [00:00<?, ? examples/s]

In [21]:
# Neste primeiro caso não faremos o monitoramento
# import os
# os.environ["WANDB_DISABLED"] = "true"

In [22]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,390,403 | Num Epochs = 1 | Total steps = 80
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.771100
2,2.968300
3,2.632800
4,2.681500
5,2.369700
6,2.220700
7,2.201700
8,2.157100
9,2.068400
10,1.668600


In [23]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "PLEASE PROVIDE ME A DESCRIPTION FOR THIS PRODUCT TITLE", # instruction
        "Girls Ballet Tutu Neon Pink", # input
        "", #output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
PLEASE PROVIDE ME A DESCRIPTION FOR THIS PRODUCT TITLE

### Input:
Girls Ballet Tutu Neon Pink

### Response:
This is a beautiful tutu in a vibrant pink color. It has a very full skirt and is very soft and comfortable. The tutu is made with a satin lining and a tulle skirt. It has a drawstring waist for easy wear and a pink satin bow. It is very easy to wear and looks beautiful on any little girl. It is made with a satin lining and a tulle skirt. It is perfect for ballet, dance, or any special occasion. It is made of a very soft and comfortable fabric. The tutu has a drawstring waist for easy wear and a pink satin bow. It is very easy to


In [24]:
# Salvando modelo treinado

model.save_pretrained("/content/drive/MyDrive/FIAP/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/lora_model")

('/content/drive/MyDrive/FIAP/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/lora_model/tokenizer.json')

In [26]:
# Testando modelo treinado salvo
import json
news_test = []

with open('/content/drive/MyDrive/FIAP/formatted_amazon_titles_search_data.json', 'r') as news_file:
  news_test = json.loads(news_file.read())['input']

In [27]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/FIAP/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "PLEASE PROVIDE ME A DESCRIPTION FOR THIS PRODUCT TITLE",
        news_test[100],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
PLEASE PROVIDE ME A DESCRIPTION FOR THIS PRODUCT TITLE

### Input:
The Long Ships: A Saga of the Viking Age

### Response:
This is the story of the great Viking kings, Ragnar Lodbrok, Bjorn Ironside, and Harald Hardrada. It is the story of their great raids on England and France, of their battles with the Franks, the Saxons, and the Welsh. It is a story of heroes and heroines, of love and hate, of faith and treachery, and of the clash of nations and civilizations.<|end_of_text|>
